<a href="https://colab.research.google.com/github/mars-ef/Trustpilot_Scraper/blob/main/Trustpilot_Web_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [80]:
#Analytics with Adam video on how to use https://www.youtube.com/watch?v=VD0wIiLVSRA
#Original code: https://github.com/analyticswithadam/Python/blob/main/Web_Scraping_from_TrustPilot.ipynb

from time import sleep
import requests
from google.colab import files
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

def soup2dict(src, data, key, attr=None):
    if attr:
        for i, val in enumerate(src):
            data[i][key] = val[attr]
    else:
        for i, val in enumerate(src):
            data[i][key] = val.get_text()

data = []
from_page = 1
to_page = 1
company = 'www.revolut.com' #E.g. for https://www.trustpilot.com/review/wise.com to sort default take 'wise.com'. To filter by most recent include 'wise.com?sort=recency'

for i in range(from_page, to_page + 1):
    result = requests.get(f"https://www.trustpilot.com/review/{company}?page={i}") #for most recent change to '{company}page={i}&sort=recency'
    soup = BeautifulSoup(result.content, 'html.parser')

    for e in soup.select('article'):
        data.append({
            'brand': "Revolut", #Add what brand here
            'username':e.select_one('[data-consumer-name-typography]').text,
            'location':e.select_one('[data-consumer-country-typography] span').text,
            'number_of_reviews':e.select_one('[data-consumer-reviews-count]').text if e.select_one('[data-consumer-reviews-count]') else None,
            'review_rating':e.select_one('[data-service-review-rating] img').get('alt'),
            'date':e.select_one('[data-service-review-rating] time').get('datetime').split('T')[0],
            'review_title':e.h2.text,
            'review_text':e.select_one('[data-service-review-text-typography]').text if e.select_one('[data-service-review-text-typography]') else None,
            'date_of_experience':e.select_one('[data-service-review-date-of-experience-typography]').text.split(': ')[-1],
            'reply_from_business':e.select_one('[data-service-review-business-reply-text-typography]').text if e.select_one('[data-service-review-business-reply-text-typography]') else None,
            'reply_date':e.select_one('[data-service-review-business-reply-date-time-ago]').text if e.select_one('[data-service-review-business-reply-date-time-ago]') else None
        })

    #To prevent throttling
    sleep(1)

# Create the DataFrame
review_data = pd.DataFrame(data)

#Formatting fixes and date standardisation
review_data['number_of_reviews'] = review_data['number_of_reviews'].str.split(' ').str[0]
review_data['review_rating'] = review_data['review_rating'].str.split(' ').str[1]
review_data['date_of_experience'] = pd.to_datetime(review_data['date_of_experience']).dt.strftime('%Y-%m-%d')

def convert_relative_date(date_str):
  if date_str is None:
    return None
  elif date_str == 'Just now':
    return datetime.now().strftime('%Y-%m-%d')
  elif date_str == 'An hour ago':
    return (datetime.now() - timedelta(hours=1)).strftime('%Y-%m-%d')
  elif date_str == 'A day ago':
    return (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')
  elif 'days ago' in date_str:
    days = int(date_str.split(' ')[0])
    return (datetime.now() - timedelta(days=days)).strftime('%Y-%m-%d')
  elif 'hours ago' in date_str:
    hours = int(date_str.split(' ')[0])
    return (datetime.now() - timedelta(hours=hours)).strftime('%Y-%m-%d')
  elif 'minutes ago' in date_str:
    minutes = int(date_str.split(' ')[0])
    return (datetime.now() - timedelta(minutes=minutes)).strftime('%Y-%m-%d')
  else:
    try:
      return datetime.strptime(date_str, '%B %d, %Y').strftime('%Y-%m-%d')
    except ValueError:
      return date_str

review_data['reply_date'] = review_data['reply_date'].apply(convert_relative_date)

In [81]:
review_data

,brand,username,location,number_of_reviews,review_rating,date,review_title,review_text,date_of_experience,reply_from_business,reply_date
0,Revolut,Moeno Mokati,FI,1,5,2024-09-25,When issues pop up,"When issues pop up, it’s really important to g...",2024-09-24,None,None
1,Revolut,Nnabiko Augustine Chime,GB,2,5,2024-09-24,I'm extremely satisfied with Revolut,I'm extremely satisfied with Revolut! I've had...,2024-09-24,None,None
2,Revolut,Dave D,GB,17,5,2024-09-25,An example to the rest in how to do it,We have been using revolut for some time now (...,2024-09-25,None,None
3,Revolut,Uma Purohit,GB,13,5,2024-09-24,One card does it all,Whilst travelling I find it easy to make payme...,2024-09-24,None,None
4,Revolut,Michael Germitsiotis,GR,1,5,2024-09-24,Excellent!,I'm very satisfied with both the day to day ex...,2024-09-24,None,None
5,Revolut,Hung Lieu,NL,1,2,2024-09-29,Recently used the product for currency…,Recently used the product for currency exchang...,2024-09-29,None,None
6,Revolut,Mārtiņš Kronbergs,LV,1,4,2024-09-24,For Ultra plan,"For Ultra plan, better to have unlimited eSim ...",2024-09-24,None,None
7,Revolut,Sylvia Ugah,GB,1,5,2024-09-24,My experience with revolut has been…,My experience with Revolut has been smooth and...,2024-09-24,None,None
8,Revolut,Ruth Hamblin,GB,1,5,2024-09-25,I rely on using my Revolut card in the…,I rely on using my Revolut card in the UK and ...,2024-09-24,None,None
9,Revolut,Karen Pugh,AU,5,4,2024-09-25,Great to use aboard,"Great to use aboard, although would like more ...",2024-09-24,None,None


In [ ]:
# Convert the DataFrame to a CSV file and download it
review_data.to_csv('reviews.csv', index=False)
files.download('reviews.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>